In [1]:
#conda install -c conda-forge wikipedia

In [2]:
# Import packages
import wikipedia
import pandas as pd
import re #Regular expressions

ModuleNotFoundError: No module named 'wikipedia'

In [3]:
import pathlib
pathlib.Path('./test_data').mkdir(parents=True, exist_ok=True)
pathlib.Path('./output').mkdir(parents=True, exist_ok=True)

In [109]:
keywords=["audit","COSO commmittee", "internal control","analytics"]
results=[]
for k in keywords:
    r = wikipedia.search(k)
    results.extend(r)
results= list(set(results))
results

['News analytics',
 'Audit trail',
 'The Audit',
 'Entity-level controls',
 'Web analytics',
 'Audit working papers',
 'Certified Internal Control Auditors',
 'Business analytics',
 'Learning analytics',
 'Financial audit',
 'Internal audit',
 'Internal control',
 'Controlled internal drug release',
 'Predictive analytics',
 'Audit (disambiguation)',
 'Audit committee',
 'Analytics',
 'Audit plan',
 'Committee of Sponsoring Organizations of the Treadway Commission',
 'Institute of Internal Auditors',
 'Verisk Analytics',
 'Google Analytics',
 'Analytic',
 'Locus of control',
 'Information technology controls',
 'Audit',
 'Audit evidence',
 'Prescriptive analytics',
 'Internal model (motor control)',
 'Information technology audit']

In [116]:
articles=[]
cap=5 #number of articles to retrieve in total
n=0
for t in results:  
    print(t)
    try:
        wiki = wikipedia.page(t)
          # Extract the plain text content of the page, excluding images, tables, and other data.
        text = wiki.content
        # Replace '==' with '' (an empty string)
        text = text.replace('==', '')
        text = text.replace('@', '')
        # Replace '\n' (a new line) with '' & end the string at $1000.
        text = text.replace('\n', '')[:-12]
        
        articles.append([t, text,wiki.url])
        n=n+1 # counting the pages received
        if n>cap:
            print("Reached the cap for downloading articles")
            break
    except wikipedia.exceptions.DisambiguationError as e:
        options= e.options
        for o in options:
            if o not in results:
                results.append(o)
    except Exception:
        print("Other error while trying to read page: "+ t)

News analytics
Audit trail
The Audit
Entity-level controls
Web analytics
Audit working papers
Reached the cap for downloading articles


In [111]:
def split_into_sentences(text):
    alphabets= "([A-Za-z])"
    suffixes = "(Inc|Ltd|Jr|Sr|Co)"
    starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
    acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
    prefixes = "(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt)[.]"
    websites = "[.](com|net|org|io|gov|me|edu|co|uk)"
    digits = "([0-9])"
    text = " " + text + "  "
    text = text.replace("\n","<stop> ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub("(\w*)[.](\w+@)","\\1<prd>\\2",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","<excl>\"")
    if "?" in text: text = text.replace("?\"","<quest>\"")
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "e.g." in text: text = text.replace("e.g.","e<prd>g<prd>") 
    if "i.e." in text: text = text.replace("i.e.","i<prd>e<prd>")
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    text = text.replace("<excl>","!")
    text = text.replace("<quest>","?")
    text = text.replace("<stop><stop>","<stop>")
    sentences = text.split("<stop>")
    #sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [112]:
sentences=[]
page_title=[]
page_url=[]
for a in articles:
    s=split_into_sentences(a[1])
    t= [a[0]] * len(s)
    u= [a[2]] * len(s)
    sentences.extend(s)
    page_title.extend(t)
    page_url.extend(u)
len(sentences)

4247

In [113]:
concat=list(zip(page_title,sentences,page_url))
df = pd.DataFrame(concat,columns=['page_title','article_sentences','page_url'])
df.head()

,page_title,article_sentences,page_url
0,News analytics,News analysis refers to the measurement of the...,https://en.wikipedia.org/wiki/News_analytics
1,News analytics,"Some of these attributes are: sentiment, relev...",https://en.wikipedia.org/wiki/News_analytics
2,News analytics,Expressing news stories as numbers and metadat...,https://en.wikipedia.org/wiki/News_analytics
3,News analytics,This data is often used in financial markets a...,https://en.wikipedia.org/wiki/News_analytics
4,News analytics,News analytics are usually derived through aut...,https://en.wikipedia.org/wiki/News_analytics


In [114]:
df.shape

(4247, 3)

In [115]:
df.to_csv("./test_data/data.csv")